In [13]:
# %%writefile 'Colltable-from-timber.py'
import time
import pytimber
import pyoptics as opt
import numpy as np
import csv

mdb=pytimber.LoggingDB(source='mdb')

# filln=5284
filln=5146
beam=1

OpticsDir = '/afs/cern.ch/user/n/nbiancac/ln_work/scratch0/IRIS/LHC_IW_model/Optics/2016/LHC-HighBeta/' # path for reference optics
CollDir='/afs/cern.ch/user/n/nbiancac/ln_work/scratch0/IRIS/LHC_IW_model/Coll_settings/'
name = 'LHC_ft_6.5TeV_B1_2016' #name of a recent collimator file
newname = 'LHC_highbeta_6.5TeV_B1'
# time_str = '2016-09-07_20:20:00'
time_str = '2016-07-31_19:40:00' # 


scenario_scan=[{'time' : time_str.replace('_',' ')}]

tfs = opt.optics.open((OpticsDir+'twiss_lhcb%d.tfs')%beam)
    
for scenario in scenario_scan:    
    
    en=3.5e-6 # reference normalized emittance
    gamma=tfs.param['gamma']; 
    
    print 'emittance = %.1f'%(en*1e6), 'gamma = %d'%gamma
        
    comment='fill_'+str(filln)+'_'+time_str
    
    reader = csv.DictReader(open(CollDir+name+'.txt'),delimiter='\t')
    lista=[];
    for row in reader:
        newrow=row.copy();

        phi=eval(row['Angle[rad]'])
        nsig=eval(row['nsig'])
        
        if 'TDI' in row['Name'] and beam==2:
            coll= mdb.search('TDI.4R8:MEAS_LVDT_GD')[0]
            bety= tfs.bety[tfs.pattern('TDI.4R8.B2')]
            betx= tfs.betx[tfs.pattern('TDI.4R8.B2')]
        
        elif 'TDI' in row['Name'] and beam==1:
            coll= mdb.search('TDI.4L2:MEAS_LVDT_GD')[0]
            bety= tfs.bety[tfs.pattern('TDI.4L2.B1')]
            betx= tfs.betx[tfs.pattern('TDI.4L2.B1')]
        
        elif 'TCDQ' in row['Name']:
            coll= mdb.search('TCDQA.%B'+str(beam)+':MEAS_LVDT_LD')[0]
            bety= tfs.bety[tfs.pattern(row['Name']+str(beam))]
            betx= tfs.betx[tfs.pattern(row['Name']+str(beam))]
        
        else:
            coll= mdb.search(row['Name']+'%MEAS_LVDT_GD')[0]
            bety= tfs.bety[tfs.pattern(row['Name'])]
            betx= tfs.betx[tfs.pattern(row['Name'])]
        
        
        data=mdb.get(coll,scenario['time'])
        tt,fullgap=data[coll]; tt+=2*3600
        hgap_new=fullgap/2*1e-3;
        
        newrow['Halfgap[m]']='%.5f'%(hgap_new);
        sigma = np.sqrt(en/gamma*betx*np.cos(phi)**2+en/gamma*bety*np.sin(phi)**2)
        nsig = hgap_new/sigma
        newrow['nsig']='%.3f'%(nsig);
        lista.append(newrow);
        print row['Name'], str(hgap_new)+' m', str(betx)+' m', str(bety)+' m \n',
    name_converted=newname+'_'+comment+'.txt'
    
    with open(CollDir+name_converted, 'wb') as f:
        # Assuming that all dictionaries in the list have the same keys.
        headers = lista[0].keys()
        csv_data = [headers];
        for d in lista:
            csv_data.append([d[h] for h in headers])
        writer = csv.writer(f,delimiter='\t')
        writer.writerows(csv_data)
    print name+' converted:'
    print name_converted

emittance = 3.5 gamma = 6927
TCTPH.4L2.B1 [ 0.01502375] m [ 44.44280984] m [ 38.50026744] m 
TCTPV.4L2.B1 [ 0.01501954] m [ 46.48637843] m [ 36.43416934] m 
TDI.4L2.B1.1 [ 0.0550911] m [ 125.25824067] m [ 47.49750576] m 
TDI.4L2.B1.1 [ 0.0550911] m [ 125.25824067] m [ 47.49750576] m 
TDI.4L2.B1.1 [ 0.0550911] m [ 125.25824067] m [ 47.49750576] m 
TDI.4L2.B1.2 [ 0.0550911] m [ 125.25824067] m [ 47.49750576] m 
TDI.4L2.B1.2 [ 0.0550911] m [ 125.25824067] m [ 47.49750576] m 
TDI.4L2.B1.3 [ 0.0550911] m [ 125.25824067] m [ 47.49750576] m 
TCLIA.4R2 [ 0.02793387] m [ 58.82592948] m [ 148.55429391] m 
TCLIB.6R2.B1 [ 0.02488027] m [ 284.00398068] m [ 29.56652712] m 
TCP.6L3.B1 [ 0.00384126] m [ 141.86555269] m [ 126.39209086] m 
TCSG.5L3.B1 [ 0.00299251] m [ 51.61327294] m [ 252.43030972] m 
TCSG.4R3.B1 [ 0.0020251] m [ 31.06863946] m [ 422.29129762] m 
TCSG.A5R3.B1 [ 0.0027136] m [ 41.02469594] m [ 369.66860783] m 
TCSG.B5R3.B1 [ 0.00302158] m [ 50.29902818] m [ 337.61891373] m 
TCLA.A5R3.B1